In [1]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import UpSampling2D
from keras.layers import Concatenate
from keras.layers import Lambda 
from keras.layers import Reshape
import tensorflow as tf

from keras import regularizers, optimizers

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
import scipy.io as scio
import numpy as np    
import os
import matplotlib.pyplot as plt
from scipy.misc import imsave
from scipy import ndimage, misc
from numpy import unravel_index
from operator import sub

In [3]:
images = []
for root, dirnames, filenames in os.walk("Dataset/Train_images/"):
    for filename in filenames:
        filepath = os.path.join(root, filename)
        image = ndimage.imread(filepath, mode="L")
        images.append(image)

In [4]:
print (images[0].shape)

(496, 64)


In [5]:
labels = np.load('train_labels.npy')
labels_list = []
for i in range(len(labels)):
    labels_list.append(labels[i])

In [6]:
labels_list[0].shape

(496, 64, 8)

In [7]:
print (len(labels_list))
print (type(labels_list[0]))

images=np.array(images)
print(images.shape[0])
images = images.reshape(images.shape[0],496,64,1)

#Increased the Size of training data
print(images[0].shape)
train_indices = np.random.choice(770,700,replace = False)
print(sorted(train_indices))
train_images = []
train_labels = [] 
for i in train_indices:
    train_images.append(images[i])
    train_labels.append(labels_list[i])

test_indices = [x for x in range(770) if x not in train_indices]
print(test_indices)
test_images = []
test_labels = []
#Corrected the varaible name
for i in test_indices:
    test_images.append(images[i])
    test_labels.append(labels_list[i])

770
<type 'numpy.ndarray'>
770
(496, 64, 1)
[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 219, 220, 221, 222, 224, 225, 226, 227, 228, 231, 232,

In [8]:
print (len(train_images))
print (len(train_labels))
print (np.array(train_labels).shape)

700
700
(700, 496, 64, 8)


In [9]:
train_images = np.array(train_images)
test_images = np.array(test_images)

In [10]:
mean = np.mean(train_images,axis=(0,1,2,3))
std = np.std(train_images,axis=(0,1,2,3))
train_images = (train_images-mean)/(std+1e-7)
test_images = (test_images-mean)/(std+1e-7)

In [11]:
train_labels = np.array(train_labels)
train_labels.shape

(700, 496, 64, 8)

In [12]:
test_labels = np.array(test_labels)
test_labels.shape

(70, 496, 64, 8)

In [10]:
data_shape = 496*64

In [14]:
train_labels = np.reshape(train_labels,(700,data_shape,8))
train_labels.shape

(700, 31744, 8)

In [15]:
#Modified the testing labels shape
test_labels = np.reshape(test_labels,(70,data_shape,8))
test_labels.shape

(70, 31744, 8)

In [4]:
double_layers = True

In [2]:
weight_decay = 1e-4

In [12]:

if double_layers :
    # Defines the input tensor
    inputs = Input(shape=(496,64,1))

    #Down Sampling Block1
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(inputs)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L1 = BatchNormalization()(L)
    L = MaxPooling2D(pool_size=(2,2))(L1)
    
    #Down Sampling Block2
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L2 = BatchNormalization()(L)
    L = MaxPooling2D(pool_size=(2,2))(L2)
    
    #Down Sampling Block3
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L3 = BatchNormalization()(L)
    L = MaxPooling2D(pool_size=(2,2))(L3)
    
    
    #Up Sampling Block1
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = UpSampling2D(size = (2,2))(L)
    
    #Skip Connection 1
    L = Concatenate(axis=3)([L,L3])
    
    #UpSampling Block2
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = UpSampling2D(size = (2,2))(L)
    
    #Skip Connection 2
    L = Concatenate(axis=3)([L,L2])
    
    #Up Sampling Block3
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = UpSampling2D(size = (2,2))(L)
    
    #Skip Connection 3
    L = Concatenate(axis = 3)([L,L1])
    
    
    #Final Conv Block
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    L = Conv2D(64,kernel_size=(3,3),activation = 'relu',padding = "same",
               kernel_regularizer=regularizers.l2(weight_decay))(L)
    L = BatchNormalization()(L)
    
    #Classifier Block
    L = Conv2D(8,kernel_size=(1,1),padding = "same")(L)
    L = Reshape((data_shape,8),input_shape = (496,64,8))(L)
    L = Activation('softmax')(L)

    #Returning Model
    model = Model(inputs = inputs, outputs = L)
    model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 496, 64, 1)    0                                            
____________________________________________________________________________________________________
conv2d_63 (Conv2D)               (None, 496, 64, 64)   640         input_7[0][0]                    
____________________________________________________________________________________________________
batch_normalization_61 (BatchNor (None, 496, 64, 64)   256         conv2d_63[0][0]                  
____________________________________________________________________________________________________
conv2d_64 (Conv2D)               (None, 496, 64, 64)   36928       batch_normalization_61[0][0]     
___________________________________________________________________________________________

In [21]:
smooth = 1

In [22]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [23]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [24]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy',dice_coef])

In [25]:
data_augmentation = True

In [26]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(train_images)
    model.fit_generator(datagen.flow(train_images, train_labels, batch_size=16),
                        steps_per_epoch=train_images.shape[0] // 16,
                        epochs=10,verbose=1,validation_data=(test_images,test_labels))
else :
    model.fit(train_images,train_labels,batch_size=16,epochs=10,validation_data=(test_images,test_labels),class_weight=class_weighting)

-------------Using Data augmentation------------
Epoch 1/10


KeyboardInterrupt: 